In [116]:
%%time
dataset_name = "W19_comb"
df_list = [ "BES_Panel" ]

%matplotlib inline 
%run BES_header.py {dataset_name} {df_list}

if "id" in BES_Panel.columns:
    BES_Panel = BES_Panel.set_index("id").sort_index()

(var_type, cat_dictionary, new_old_col_names, old_new_col_names) = get_small_files(data_subfolder, encoding)    
    
# get full set of inferred "cross wave" auth-lib/left-right values and ages
# pan_dataset_allr_values = pd.read_csv(BES_small_data_files + "pan_dataset_allr_valuesW19"+".csv")
pan_dataset_ages = pd.read_pickle(BES_small_data_files + "pan_dataset_ages"+".zip", compression='zip')

Toggle code

var_type (7911, 14)
Wall time: 29.3 s


In [318]:
# City of London,No formal qualifications = Conservative
# Cambden,Youth training certificate/skillseekers = Labour
# Greenwich,Recognised trade apprenticeship completed = Lib Dems
# Hackney,Clerical and commercial = SNP
# Hammersmith and Fulham,City & Guilds certificate = PC
# Islington,City & Guilds certificate - advanced = Green
# Kensington and Chelsea,onc = UKIP
# Lambeth,CSE grades 2-5 = Brexit Party
# Lewisham,CSE grade 1, GCE O level, GCSE, School Certificate = TIG
# Bromley,Other technical, professional or higher qualification = Other
# Ealing,Prefer not to say = Don't know

In [120]:
BES_Panelv01 = BES_Panel.copy()

In [121]:
BES_Panel = pd.read_pickle("..\\BES_analysis_data\\"+"W19_comb"+os.sep+"BES_Panelv02",compression='zip')
BES_Panel = BES_Panel.set_index("id")
BES_Panel = BES_Panel.sort_index()

In [ ]:
# scotRefTurnoutW3_W14          8031
# scotRefVoteW3_W14             7401
# profile_scotRefW17            3668
# profile_scotRefW18            4203
# profile_scotRefW19            3498
# profile_scotref_turnout       9959
# profile_scotref_vote          8799

In [267]:
BES_Panel["pastvote_eup_2019W18"] = pastvote_eup_2019W18.loc[BES_Panel.index]
BES_Panel["pastvote_eup_2019W19"] = pastvote_eup_2019W19.loc[BES_Panel.index]

BES_Panel["scotReferendumVoteW11"] = scotReferendumVoteW11.loc[BES_Panel.index]
BES_Panel["scotReferendumVoteW15"] = scotReferendumVoteW15.loc[BES_Panel.index]

fixed_eup_categories = ['Conservative', 'Labour', 'Liberal Democrat',
       'Scottish National Party (SNP)', 'Plaid Cymru',
       'Green Party','United Kingdom Independence Party (UKIP)','Brexit Party', 
       'Change UK- The Independent Group', 'Other', "Don't know"]
BES_Panel["pastvote_eup_2019W18"].cat.rename_categories(fixed_eup_categories,inplace=True)
BES_Panel["pastvote_eup_2019W19"].cat.rename_categories(fixed_eup_categories,inplace=True)

BES_Panel["pastvote_eup_2019W18"] = BES_Panel["pastvote_eup_2019W18"].cat.reorder_categories(BES_Panel["euroElectionVoteW16"].cat.categories)
BES_Panel["pastvote_eup_2019W19"] = BES_Panel["pastvote_eup_2019W19"].cat.reorder_categories(BES_Panel["euroElectionVoteW16"].cat.categories)

BES_Panel[match(BES_Panel,"countryW\d+").index] = BES_Panel[match(BES_Panel,"countryW\d+").index].replace({1:"England",2:"Scotland",3:"Wales"})

extra_vote_var_list = ['scotReferendumVoteW3', 'scotReferendumVoteW11',  'scotReferendumVoteW15']
BES_Panel[extra_vote_var_list] = BES_Panel[extra_vote_var_list].replace({"I voted 'No' (Scotland should not be an independent country)":"Voted no",
                                   "I voted 'Yes' (Scotland should be an independent country)":"Voted yes"})

# 98.0 in localElectionVoteW12 *appears to be "An independent candidate"
# guess they decided to move all these to Other ... but missed this one
BES_Panel['localElectionVoteW12'] = BES_Panel['localElectionVoteW12'].replace(98.0,"Other")

In [14]:
BES_Panel[search(BES_Panel,"p_past_vote").index].apply(lambda x: "|".join( x.cat.categories ) ).values

array(["Conservative|Labour|Liberal Democrat|Scottish National Party (SNP)|Plaid Cymru|United Kingdom Independence Party (UKIP)|Green Party|British National Party (BNP)|Other|Don't know",
       "Conservative|Labour|Liberal Democrat|Scottish National Party (SNP)|Plaid Cymru|United Kingdom Independence Party (UKIP)|Green Party|British National Party (BNP)|Other|Don't know",
       "Conservative|Labour|Liberal Democrat|Scottish National Party (SNP)|Plaid Cymru|United Kingdom Independence Party (UKIP)|Green Party|British National Party (BNP)|Other|Don't know",
       "Conservative|Labour|Liberal Democrat|Scottish National Party (SNP)|Plaid Cymru|United Kingdom Independence Party (UKIP)|Green Party|British National Party (BNP)|Other|Don't know",
       "Conservative|Labour|Liberal Democrat|Scottish National Party (SNP)|Plaid Cymru|Green Party|Other|Brexit Party|Don't know"],
      dtype=object)

In [30]:
BES_Panel["p_past_vote_2017"].value_counts()

Labour                                      23742
Conservative                                23741
Liberal Democrat                             5258
Scottish National Party (SNP)                2523
United Kingdom Independence Party (UKIP)     1498
Green Party                                  1289
Don't know                                   1204
Other                                         726
Plaid Cymru                                   395
British National Party (BNP)                    1
Name: p_past_vote_2017, dtype: int64

In [12]:
BES_Panel[search(BES_Panel,"p_turnout").index].apply(lambda x: "|".join( x.cat.categories ) )

p_turnout_2015                                     DNV|Voted|Don't know
p_turnout_2017                                     DNV|Voted|Don't know
p_turnout_2019                                     DNV|Voted|Don't know
p_turnout_2010                                                DNV|Voted
p_turnout_2005                                                DNV|Voted
p_turnout_scot_ref    No, I did not vote|Yes, I voted|No, I was not ...
dtype: object

In [65]:
mask = BES_Panel[ search(BES_Panel,"euRefpastVote").index ].replace("Don't know",np.nan).notnull().any(axis=1)

In [81]:
search(BES_Panel,"generalElectionVoteRetro|TurnoutRetro").index


# welshTurnoutRetroW8, scotTurnoutRetroW8

Index(['generalElectionVoteRetro2017W18', 'euroTurnoutRetroW2',
       'localTurnoutRetroW2', 'genElecTurnoutRetroW6', 'localTurnoutRetroW6',
       'genElecTurnoutRetroW7', 'localTurnoutRetroW8', 'welshTurnoutRetroW8',
       'scotTurnoutRetroW8', 'euRefTurnoutRetroW9', 'euRefTurnoutRetroW11',
       'localTurnoutRetroW12', 'euRefTurnoutRetroW12',
       'genElecTurnoutRetroW13', 'euRefTurnoutRetroW13',
       'localTurnoutRetroW14', 'euRefTurnoutRetroW14', 'euRefTurnoutRetroW15',
       'euroTurnoutRetroW16', 'localTurnoutRetroW16', 'euRefTurnoutRetroW17',
       'genElecTurnoutRetro2017W18', 'genElecTurnoutRetroW19'],
      dtype='object')

In [83]:
search(BES_Panel,"euroTurnout|euroElectionVote|voteMethodEurope").index

# voteMethodEurope

# welshTurnoutRetroW8, scotTurnoutRetroW8

Index(['euroTurnoutW1', 'euroElectionVoteW1', 'euroTurnoutRetroW2',
       'euroElectionVoteW2', 'voteMethodEurope_1W2', 'voteMethodEurope_2W2',
       'voteMethodEurope_3W2', 'voteMethodEurope_4W2', 'euroTurnoutRetroW16',
       'euroElectionVoteW16', 'euroElectionVoteNonVoterW16',
       'voteMethodEurope_1W16', 'voteMethodEurope_2W16',
       'voteMethodEurope_3W16', 'voteMethodEurope_4W16'],
      dtype='object')

In [79]:
search(BES_Panel,"euref").index

Index(['euRefVoteW1', 'euRefVoteW2', 'euRefVoteW3', 'euRefVoteW4',
       'euRefVoteW6', 'euRefTurnoutW7', 'euRefVoteW7', 'euRefVoteSqueezeW7',
       'euRefInterestW7', 'euRefExpectationW7', 'euRefFinalW7',
       'expectGoodConductEURefW7', 'euRefVoteUnsqueezeW7', 'euRefVotePostW8',
       'euRefTurnoutW8', 'euRefVoteW8', 'euRefVoteSqueezeW8',
       'euRefCertaintyW8', 'euRefInterestW8', 'euRefExpectationW8',
       'euRefFinalW8', 'euRefVoteUnsqueezeW8', 'euRefVote2W8',
       'euRefTurnoutRetroW9', 'euRefVoteHypotheticalW9', 'euRefVoteW9',
       'euRefFinalPostW9', 'goodConductEURefW9', 'euRefDoOverW10',
       'euRefVoteW10', 'goodConductEURefW10', 'euRefDoOverW11', 'euRefVoteW11',
       'goodConductEURefW11', 'euRefTurnoutRetroW11', 'euRefpastVoteW11',
       'euRefVoteW12', 'euRefTurnoutRetroW12', 'euRefpastVoteW12',
       'euRefDoOverW13', 'euRefVoteW13', 'euRefTurnoutRetroW13',
       'euRefpastVoteW13', 'euRefDoOverW14', 'euRefVoteW14',
       'euRefTurnoutRetroW14', 'euR

In [77]:
BES_Panel[ search(BES_Panel,"pastvote").index ][BES_Panel["p_eurefvote"].isnull()&mask].replace("Don't know",np.nan).mode(axis=1,dropna=True)[0]

id
81       Stay/remain in the EU
458               Leave the EU
544               Leave the EU
778               Leave the EU
1068              Leave the EU
                 ...          
80899             Leave the EU
80912             Leave the EU
80985    Stay/remain in the EU
81072    Stay/remain in the EU
81082    Stay/remain in the EU
Name: 0, Length: 945, dtype: object

In [97]:
list( search(BES_Panel,"turnout").index )

['turnoutUKGeneralW1',
 'turnoutUKGeneralW2',
 'turnoutUKGeneralW3',
 'turnoutUKGeneralW4',
 'turnoutUKGeneralW5',
 'turnoutUKGeneralW7',
 'turnoutUKGeneralW8',
 'turnoutUKGeneralW9',
 'turnoutUKGeneralW10',
 'turnoutUKGeneralW11',
 'turnoutUKGeneralW12',
 'turnoutUKGeneralW14',
 'turnoutUKGeneralW15',
 'turnoutUKGeneralW16',
 'turnoutUKGeneralW17',
 'turnoutUKGeneralW18',
 'euroTurnoutW1',
 'scotReferendumTurnoutW1',
 'scotRefExpectationTurnoutW1',
 'euroTurnoutRetroW2',
 'localTurnoutRetroW2',
 'discussantturnoutName1W2',
 'discussantturnoutName2W2',
 'discussantturnoutName3W2',
 'scotReferendumTurnoutW2',
 'scotRefExpectationTurnoutW2',
 'mapTurnoutW3',
 'discussantturnoutName1W4',
 'discussantturnoutName2W4',
 'discussantturnoutName3W4',
 'postalTurnoutW5',
 'genElecTurnoutRetroW6',
 'localTurnoutRetroW6',
 'euRefTurnoutW7',
 'localTurnoutW7',
 'welshTurnoutW7',
 'scotTurnoutW7',
 'londonTurnoutW7',
 'genElecTurnoutRetroW7',
 'postalTurnoutW8',
 'euRefTurnoutW8',
 'localTurnoutRetroW8',
 'welshTurnoutRetroW8',
 'scotTurnoutRetroW8',
 'pccTurnoutW8',
 'londonTurnoutW8',
 'euRefTurnoutRetroW9',
 'euRefTurnoutRetroW11',
 'postalTurnoutW12',
 'localTurnoutRetroW12',
 'discussantturnoutName1W12',
 'discussantturnoutName2W12',
 'discussantturnoutName3W12',
 'euRefTurnoutRetroW12',
 'genElecTurnoutRetroW13',
 'euRefTurnoutRetroW13',
 'localTurnoutRetroW14',
 'euRefTurnoutRetroW14',
 'euRefTurnoutRetroW15',
 'euroTurnoutRetroW16',
 'localTurnoutRetroW16',
 'euRefTurnoutRetroW17',
 'postalTurnoutW18',
 'genElecTurnoutRetro2017W18',
 'genElecTurnoutRetroW19',
 'p_turnout_2015',
 'p_turnout_2017',
 'p_turnout_2019',
 'p_turnout_2010',
 'p_turnout_2005',
 'p_turnout_scot_ref',
 'p_eurefturnout']

In [93]:
BES_Panel["pastvote_eup_2019W18"] = pastvote_eup_2019W18.loc[BES_Panel.index]
BES_Panel["pastvote_eup_2019W19"] = pastvote_eup_2019W19.loc[BES_Panel.index]

BES_Panel["scotReferendumVoteW11"] = scotReferendumVoteW11.loc[BES_Panel.index]
BES_Panel["scotReferendumVoteW15"] = scotReferendumVoteW15.loc[BES_Panel.index]

id
1        Yes, voted
2        Yes, voted
3        Yes, voted
4               NaN
5               NaN
            ...    
98447           NaN
98448           NaN
98449           NaN
98450           NaN
98451           NaN
Name: genElecTurnoutRetroW6, Length: 97006, dtype: category
Categories (3, object): [No, did not vote < Yes, voted < Don't know]

In [ ]:
# voting events

# GE 2005,2010,2015,2017,2019 DONE
# euRefTurnoutRetro W9W11W12W13W14W15W17
# euRefpastVote W11W12W13W15
# p_eurefturnout
# p_eurvote DONE

# euroTurnoutRetro W2W16
# euroElectionVote W2W16
# pastvote_eup_2019 W18W19 DONE


# SCOTREF, EUREF
# EURO 2014,2019


# voteMethodEurope_(1|2|3|4|dk) W2W16 (same coverage as euroElectionVote)
# euroElectionVoteNonVoterW16  (good coverage of *all* the people not registered as voting!)


# scotReferendumVote W11W15
# scotReferendumTurnout W1W2
# scotReferendumRetroW3
# scotReferendumVoteW3
p_vote_scot_ref                   9588
p_turnout_scot_ref               10976 DONE


# localTurnoutRetro W2W8W12W14W16
# localElectionVote W2W8W12W14W16 DONE



p_scot_const_vote_2011            9581
p_scot_list_vote_2011             9489

p_scot_const_vote_2016            8637
p_scot_list_vote_2016             8635
# scotTurnoutRetroW8

p_welsh_const_vote_2016    5212
p_welsh_list_vote_2016     5211
# welshTurnoutRetroW8

# <specific mayoral votes>

In [387]:
search(BES_Panel,"londonTurnout")

londonTurnoutW7    3598
londonTurnoutW8    3702
dtype: int64

In [140]:
BES_Panel[["p_turnout_2005","p_turnout_2010","p_turnout_2015","p_turnout_2017","p_turnout_2019",
           "voted2015newwordingW15","genElecTurnoutRetroW6","genElecTurnoutRetroW7",
           "genElecTurnoutRetroW13","genElecTurnoutRetro2017W18",
           "genElecTurnoutRetroW19"]].apply(lambda x:"|".join(x.cat.categories))

p_turnout_2005                                             DNV|Voted
p_turnout_2010                                             DNV|Voted
p_turnout_2015                                  DNV|Voted|Don't know
p_turnout_2017                                  DNV|Voted|Don't know
p_turnout_2019                                  DNV|Voted|Don't know
voted2015newwordingW15        No, did not vote|Yes, voted|Don't know
genElecTurnoutRetroW6         No, did not vote|Yes, voted|Don't know
genElecTurnoutRetroW7         No, did not vote|Yes, voted|Don't know
genElecTurnoutRetroW13        No, did not vote|Yes, voted|Don't know
genElecTurnoutRetro2017W18    No, did not vote|Yes, voted|Don't know
genElecTurnoutRetroW19        No, did not vote|Yes, voted|Don't know
dtype: object

In [468]:
unknown_str = "UNK"
tooYoung_str = "TYTV(Def)"
tooYoung_soft_str = "TYTV(Maybe)"
noWeights_str = "Not Present"
nonVoter_str = "DNV"
couldNotVote_str = "CNV"
age_cut_off = 18

replace_dict = {'Conservative':"CON",
 'Labour':"LAB",
 'Liberal Democrat':"LD",
 'Scottish National Party (SNP)':"SNP",
 'Plaid Cymru':"PC",
 'Green Party':"GP",
 'Other':"OTH",
 'Brexit Party':"BXP",
 "Don't know":"DK",
 "United Kingdom Independence Party (UKIP)":"UKIP",
 "British National Party (BNP)":"BNP",
 "Stay/remain in the EU":"REMAIN",
 "Leave the EU":"LEAVE",
 "Change UK- The Independent Group":"TIG",
 "Voted no":"NO","Voted yes":"YES",
 "I voted 'No' (Scotland should not be an independent country)":"NO",
 "I voted 'Yes' (Scotland should be an independent country)":"YES",
 "I would/did not vote":nonVoter_str,
               }



Probably_didnt_vote_list = ["No, did not vote","No, I did not vote","DNV","I would/did not vote",
                            "No, I was not eligible to vote","There wasn't a local election in my area"]

Couldnt_vote_list = ["No, I was not eligible to vote","There wasn't a local election in my area"]

nowtsW19 = (~BES_Panel[match(BES_Panel,"wave\d+$").index].any(axis=1))

dk_replace = {"Don't know":np.nan,"DK":np.nan}

In [489]:



def add_cat_and_modify_df(pan_dataset_votes, col, mask, new_cat ):
    if new_cat not in pan_dataset_votes[col].cat.categories:
        pan_dataset_votes[col].cat.add_categories(new_cat,inplace=True)
    pan_dataset_votes[col][mask] = new_cat
    
def were_they_too_young_to_vote(age_var,offset):
    too_young_to_vote_hard = pan_dataset_ages[age_var]<=(age_cut_off+offset-1)
    too_young_to_vote_soft = pan_dataset_ages[age_var]<=(age_cut_off+offset+1)
    return too_young_to_vote_hard,too_young_to_vote_soft

def update_pan_dataset_votes(turnout_var,past_vote_var,new_vote_var,age_var,age_offset=0,
                             extra_turnout_var_list=[],extra_vote_var_list=[],could_not_vote_var_list=[],
                             could_not_vote_extras=[]):
    pan_dataset_votes[new_vote_var] = BES_Panel[past_vote_var].copy()
    # BES_Panel["profile_past_vote_2005"].cat.categories

    # 'An independent candidate' -> An independent candidate
    # BES_Panel["profile_past_vote_2005"].replace(replace_dict)

    for extra_vote_var in extra_vote_var_list:
        no_confident_vote_inf = pan_dataset_votes[new_vote_var].replace(dk_replace).isnull()
        pan_dataset_votes.loc[no_confident_vote_inf,new_vote_var] = BES_Panel.loc[no_confident_vote_inf,extra_vote_var]
        
    pan_dataset_votes[new_vote_var] = pan_dataset_votes[new_vote_var].replace(replace_dict).astype('category')

    # Did not vote
    if turnout_var is not None:
        Didnt_vote_not_sure = BES_Panel[turnout_var].apply(lambda x: x in Probably_didnt_vote_list).fillna(False).astype('bool')
        for extra_turnout_var in extra_turnout_var_list:
            Didnt_vote_not_sure = Didnt_vote_not_sure|BES_Panel[extra_turnout_var].apply(lambda x: x in Probably_didnt_vote_list).replace(np.nan,False)   
        # non profile variables only contribute where profile variables is not present
        no_confident_vote_inf = pan_dataset_votes[new_vote_var].replace(dk_replace).isnull()
        Didnt_vote_not_sure = Didnt_vote_not_sure & no_confident_vote_inf
        add_cat_and_modify_df(pan_dataset_votes, new_vote_var, Didnt_vote_not_sure, nonVoter_str)            

    # Could not vote
    # check turnout variables for "couldnt_vote_list" strings
    Could_not_vote = pd.Series([False]*BES_Panel.shape[0],index=BES_Panel.index)
    if turnout_var is not None:
        Could_not_vote =  Could_not_vote|BES_Panel[turnout_var].apply(lambda x: x in Couldnt_vote_list+could_not_vote_extras).fillna(False).astype('bool')
        for extra_turnout_var in extra_turnout_var_list:
            Could_not_vote = Could_not_vote|BES_Panel[extra_turnout_var].apply(lambda x: x in Couldnt_vote_list+could_not_vote_extras).replace(np.nan,False)
    for could_not_vote_var in could_not_vote_var_list:
        Could_not_vote = Could_not_vote|BES_Panel[could_not_vote_var].apply(lambda x: x in Couldnt_vote_list+could_not_vote_extras).replace(np.nan,False)
    
    if Could_not_vote.sum()>0:
        add_cat_and_modify_df(pan_dataset_votes, new_vote_var, Could_not_vote, couldNotVote_str)
    

    # could break this into two phases
    # Age < CutOff-1 => Confident enough to override recollections of voting!
    # Age < Cutoff+1 => Only overrides Don't know/absence of info
    too_young_to_vote_hard,too_young_to_vote_soft = were_they_too_young_to_vote(age_var,age_offset)
    add_cat_and_modify_df(pan_dataset_votes, new_vote_var, too_young_to_vote_hard, tooYoung_str)

    no_confident_vote_inf = pan_dataset_votes[new_vote_var].replace(dk_replace).isnull()
    too_young_to_vote_soft = too_young_to_vote_soft & no_confident_vote_inf
    add_cat_and_modify_df(pan_dataset_votes, new_vote_var, too_young_to_vote_soft, tooYoung_soft_str)

    pan_dataset_votes[new_vote_var+"_complete"] = pan_dataset_votes[new_vote_var]
    # entried that don't actually seem real/to contain any data!
    # nowtsW19 = (BES_Panel["wt_new_W19_result"].isnull() & BES_Panel["wave19"])
    add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", nowtsW19, noWeights_str)

    # we have no information - probably because not covered in any wave
    unknownW19 = pan_dataset_votes[new_vote_var+"_complete"].isnull()
    add_cat_and_modify_df(pan_dataset_votes, new_vote_var+"_complete", unknownW19, unknown_str)   
    
    pan_dataset_votes[new_vote_var+"_simple"] = pan_dataset_votes[new_vote_var+"_complete"]
    pan_dataset_votes[new_vote_var+"_simple"].replace( {"Not Present":np.nan,"TYTV(Def)":"TYTV","TYTV(Maybe)":"TYTV" },inplace=True )
    pan_dataset_votes[new_vote_var+"_simple"] = pan_dataset_votes[new_vote_var+"_simple"].astype('category')

In [490]:
    
pan_dataset_votes = pd.DataFrame()
    
turnout_var = "p_turnout_2005"
past_vote_var = "p_past_vote_2005"
new_vote_var = "2005GEVote"
age_var = "Age_at_W1"
age_offset =8.818617385352498
update_pan_dataset_votes(turnout_var,past_vote_var,new_vote_var,age_var,age_offset)    

turnout_var = "p_turnout_2010"
past_vote_var = "p_past_vote_2010"
new_vote_var = "2010GEVote"
age_var = "Age_at_W1"
age_offset =3.818617385352498
update_pan_dataset_votes(turnout_var,past_vote_var,new_vote_var,age_var,age_offset)    

turnout_var = "p_turnout_2015"
past_vote_var = "p_past_vote_2015"
new_vote_var = "2015GEVote"
age_var = "Age_at_W5"
extra_turnout_var_list = ["voted2015newwordingW15","genElecTurnoutRetroW6","genElecTurnoutRetroW7"]
update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,age_var=age_var,
                         extra_turnout_var_list=extra_turnout_var_list) 

turnout_var = "p_turnout_2017"
past_vote_var = "p_past_vote_2017"
new_vote_var = "2017GEVote"
age_var = "Age_at_W12"
extra_turnout_var_list = ["genElecTurnoutRetroW13","genElecTurnoutRetro2017W18"]
update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,age_var=age_var,
                         extra_turnout_var_list=extra_turnout_var_list) 

turnout_var = "p_turnout_2019"
past_vote_var = "p_past_vote_2019"
new_vote_var = "2019GEVote"
age_var = "Age_at_W18"
extra_turnout_var_list = ["genElecTurnoutRetroW19"]
update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,age_var=age_var,
                         extra_turnout_var_list=extra_turnout_var_list) 

pan_dataset_votes["2019GEVote_simple"].value_counts()

UNK     64745
CON     12695
LAB      8264
LD       3582
DNV      3224
SNP      1252
GP        863
BXP       722
OTH       552
TYTV      226
DK        221
PC        183
Name: 2019GEVote_simple, dtype: int64

In [491]:
turnout_var = "p_eurefturnout"
past_vote_var = "p_eurefvote"
new_vote_var = "2016EUVote"
age_var = "Age_at_W8"
extra_turnout_var_list = ['euRefTurnoutRetroW9', 'euRefTurnoutRetroW11', 'euRefTurnoutRetroW12',
       'euRefTurnoutRetroW13', 'euRefTurnoutRetroW14', 'euRefTurnoutRetroW15',
       'euRefTurnoutRetroW17']
extra_vote_var_list = ['euRefpastVoteW11', 'euRefpastVoteW12', 'euRefpastVoteW13',
       'euRefpastVoteW15']

update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,age_var=age_var,
                         extra_turnout_var_list=extra_turnout_var_list,
                         extra_vote_var_list = extra_vote_var_list) 



turnout_var = "euroTurnoutRetroW2"
past_vote_var = "euroElectionVoteW2"
new_vote_var = "2014EEVote"
age_var = "Age_at_W1"
update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,age_var=age_var) 


turnout_var = "euroTurnoutRetroW16"
past_vote_var = "euroElectionVoteW16"
new_vote_var = "2019EEVote"
age_var = "Age_at_W16"
extra_vote_var_list = [ 'pastvote_eup_2019W18', 'pastvote_eup_2019W18' ]
update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,age_var=age_var,
                         extra_vote_var_list = extra_vote_var_list) 



pan_dataset_votes[new_vote_var+"_simple"].value_counts()


UNK     48627
BXP     12733
DNV     11523
LD       7536
GP       4812
LAB      3822
CON      2487
SNP      1485
TIG      1197
OTH       708
UKIP      552
PC        472
DK        311
TYTV      264
Name: 2019EEVote_simple, dtype: int64

In [492]:
turnout_var = "p_turnout_scot_ref"
past_vote_var = "p_vote_scot_ref"
new_vote_var = "2014SCOTREF"
age_var = "Age_at_W2"
age_offset = -2
extra_vote_var_list = ['scotReferendumVoteW3', 'scotReferendumVoteW11',  'scotReferendumVoteW15']

extra_turnout_var_list = ['scotReferendumRetroW3']

update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,
                         age_var=age_var,age_offset = age_offset,
                         extra_turnout_var_list=extra_turnout_var_list,
                         extra_vote_var_list = extra_vote_var_list,
                         could_not_vote_var_list=["countryW2"],could_not_vote_extras=["England","Wales"]) 

pan_dataset_votes[new_vote_var+"_simple"].value_counts()




                             

UNK     57523
CNV     24273
NO       5030
YES      4480
TYTV     4389
DNV       832
DK          2
Name: 2014SCOTREF_simple, dtype: int64

In [476]:
# UK local elections
# https://en.wikipedia.org/wiki/2019_United_Kingdom_local_elections
    
# * 2 May 2019	W16
# * 3 May 2018	W14
# * 4 May 2017	W12
# * 5 May 2016 - W8
# * 7 May 2015 - W6
# * 22 May 2014   - W2

In [493]:
turnout_var = "localTurnoutRetroW2"
past_vote_var = "localElectionVoteW2"
new_vote_var = "2014LE"
age_var = "Age_at_W2"

update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,age_var=age_var) 

turnout_var = "localTurnoutRetroW6"
past_vote_var = "localElectionVoteW6"
new_vote_var = "2015LE"
age_var = "Age_at_W6"

update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,age_var=age_var) 

turnout_var = "localTurnoutRetroW8"
past_vote_var = "localElectionVoteW8"
new_vote_var = "2016LE"
age_var = "Age_at_W8"

update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,age_var=age_var) 

turnout_var = "localTurnoutRetroW12"
past_vote_var = "localElectionVoteW12"
new_vote_var = "2017LE"
age_var = "Age_at_W12"


update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,age_var=age_var) 

turnout_var = "localTurnoutRetroW14"
past_vote_var = "localElectionVoteW14"
new_vote_var = "2018LE"
age_var = "Age_at_W14"

update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,age_var=age_var)

turnout_var = "localTurnoutRetroW16"
past_vote_var = "localElectionVoteW16"
new_vote_var = "2019LE"
age_var = "Age_at_W16"

update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,age_var=age_var)

pan_dataset_votes[new_vote_var+"_simple"].value_counts()


UNK     73126
DNV      7609
CNV      3203
CON      3168
LAB      2872
LD       2212
OTH      1713
GP       1140
UKIP      701
TYTV      447
DK        338
Name: 2019LE_simple, dtype: int64

In [494]:
turnout_var = None
past_vote_var = "p_scot_const_vote_2011"
new_vote_var = "2011SEC"
age_var = "Age_at_W1"
age_offset = 2.818617385352498

update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,
                         age_var=age_var,age_offset=age_offset) 



turnout_var = None
past_vote_var = "p_scot_list_vote_2011"
new_vote_var = "2011SEL"
age_var = "Age_at_W1"
age_offset = 2.818617385352498

update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,
                         age_var=age_var,age_offset=age_offset) 



turnout_var = "scotTurnoutRetroW8"
past_vote_var = "p_scot_const_vote_2016"
new_vote_var = "2016SEC"
age_var = "Age_at_W7"
age_offset = -2


update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,
                         age_var=age_var,age_offset=age_offset,
                         could_not_vote_var_list=["countryW7"],could_not_vote_extras=["England","Wales"]) 


turnout_var = "scotTurnoutRetroW8"
past_vote_var = "p_scot_list_vote_2016"
new_vote_var = "2016SEL"
age_var = "Age_at_W7"
age_offset = -2

update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,
                         age_var=age_var,age_offset=age_offset,
                         could_not_vote_var_list=["countryW7"],could_not_vote_extras=["England","Wales"]) 


turnout_var = "welshTurnoutRetroW8"
past_vote_var = "p_welsh_const_vote_2016"
new_vote_var = "2016WEC"
age_var = "Age_at_W7"


update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,
                         age_var=age_var,
                         could_not_vote_var_list=["countryW7"],could_not_vote_extras=["England","Scotland"]) 

# pan_dataset_votes[new_vote_var+"_complete"].value_counts()

turnout_var = "welshTurnoutRetroW8"
past_vote_var = "p_welsh_list_vote_2016"
new_vote_var = "2016WEL"
age_var = "Age_at_W7"

update_pan_dataset_votes(turnout_var=turnout_var,past_vote_var=past_vote_var,new_vote_var=new_vote_var,
                         age_var=age_var,
                         could_not_vote_var_list=["countryW7"],could_not_vote_extras=["England","Scotland"]) 



pan_dataset_votes[new_vote_var+"_simple"].value_counts()



UNK     59351
CNV     28315
TYTV     3730
DNV      2181
LAB       897
PC        645
CON       592
UKIP      325
LD        227
GP        151
OTH       115
Name: 2016WEL_simple, dtype: int64

In [498]:
pan_dataset_votes.to_csv(BES_small_data_files+"pan_dataset_votes"+".csv")
pan_dataset_votes.to_pickle(BES_small_data_files+"pan_dataset_votes"+".zip",compression='zip')

'../BES_analysis_code/small data files\\'

In [ ]:
# No, did not vote|Yes, voted|There wasn't a local election in my area|Don't know

In [426]:
BES_Panel['p_welsh_const_vote_2016'].value_counts()

I would/did not vote                        1529
Labour                                      1279
Conservative                                 809
Plaid Cymru                                  745
United Kingdom Independence Party (UKIP)     407
Liberal Democrat                             292
Other                                        151
Name: p_welsh_const_vote_2016, dtype: int64

In [428]:
BES_Panel['welshTurnoutRetroW8'].value_counts()

Yes, voted          2152
No, did not vote     386
Don't know            33
Name: welshTurnoutRetroW8, dtype: int64

In [406]:
BES_Panel['localElectionVoteW12'].cat.categories

Index([                            'Conservative',
                                         'Labour',
                               'Liberal Democrat',
                  'Scottish National Party (SNP)',
                                    'Plaid Cymru',
       'United Kingdom Independence Party (UKIP)',
                                    'Green Party',
                                          'Other',
                                             98.0,
                                     'Don't know'],
      dtype='object')

In [433]:
BES_file_manifest

Name                                          BES_Title  \
0           W19_only  Wave 19 of the 2014-2023 British Election Stud...   
1           W19_comb  British Election Study Combined Wave 1-19 Inte...   
2   W19_comb_strings  British Election Study Combined Wave 1-19 Inte...   
3           W18_only  Wave 18 of the 2014-2023 British Election Stud...   
4           W17_only  Wave 17 of the 2014-2023 British Election Stud...   
5           W16_only  Wave 16 of the 2014-2019 British Election Stud...   
6           W16_comb  British Election Study Combined Wave 1-16 Inte...   
7   W16_comb_strings  British Election Study Combined Wave 1-16 Inte...   
8           W15_only  Wave 15 of the 2014-2019 British Election Stud...   
9           W15_comb  British Election Study Combined Wave 1-15 Inte...   
10          W14_only  Wave 14 of the 2014-2018 British Election Stud...   
11          W14_comb  British Election Study Combined Wave 1-14 Inte...   
12  W14_comb_strings  British Election Study Combined Wave 1-14 Inte...   
13          W13_only  Wave 13 of the 2014-2018 British Election Stud...   
14          W13_comb  British Election Study Combined Wave 1-13 Inte...   
15  W13_comb_strings  British Election Study Combined Wave 1-13 Inte...   
16          W12_only  Wave 12 of the 2014-2018 British Election Stud...   
17          W11_only  Wave 11 of the 2014-2018 British Election Stud...   
18          W10_comb  British Election Study Combined Wave 1-10 Inte...   
19          W10_only  Wave 10 of the 2014-2018 British Election Stud...   
20           W9_comb  British Election Study Combined Wave 1-9 Inter...   
21           W9_only  Wave 9 of the 2014-2018 British Election Study...   
22           W8_only  Wave 8 of the 2014-2018 British Election Study...   
23           W8_comb  British Election Study Combined Wave 1, 2, 3, ...   
24           W7_only  Wave 7 of the 2014-2018 British Election Study...   
25           W7_comb  British Election Study Combined Wave 1, 2, 3, ...   
26           W6_only  Post-Election Wave 6 of the 2014-2018 British ...   
27           W6_comb  British Election Study Combined Wave 1, 2, 3, ...   
28           W5_comb  British Election Study Combined Wave 1, 2, 3, ...   
29           W5_only  Wave 5 of the 2014-2018 British Election Study...   
30           W4_only  Updated Wave 4 of the 2014-2018 British Electi...   
31           W4_comb  Updated British Election Study Combined Wave 1...   
32           W3_only  Wave 3 of the 2014-2018 British Election Study...   
33           W3_comb  2015 British Election Study Combined Wave 1, 2...   
34           W2_only  Wave 2 of the 2014-2018 British Election Study...   
35           W1_only  Wave 1 of the 2014-2018 British Election Study...   

                                              BES_Url  Wave No  \
0   https://www.britishelectionstudy.com/data-obje...       19   
1   https://www.britishelectionstudy.com/data-obje...       19   
2   https://www.britishelectionstudy.com/data-obje...       19   
3   https://www.britishelectionstudy.com/data-obje...       18   
4   https://www.britishelectionstudy.com/data-obje...       17   
5   https://www.britishelectionstudy.com/data-obje...       16   
6   https://www.britishelectionstudy.com/data-obje...       16   
7   https://www.britishelectionstudy.com/data-obje...       16   
8   https://www.britishelectionstudy.com/data-obje...       15   
9   https://www.britishelectionstudy.com/data-obje...       15   
10  http://www.britishelectionstudy.com/data-objec...       14   
11  http://www.britishelectionstudy.com/data-objec...       14   
12  http://www.britishelectionstudy.com/data-objec...       14   
13  http://www.britishelectionstudy.com/data-objec...       13   
14  http://www.britishelectionstudy.com/data-objec...       13   
15  http://www.britishelectionstudy.com/data-objec...       13   
16  http://www.britishelectionstudy.com/data-objec...       12   
17  http://www.britishelectionstudy.com/data-objec..

In [426]:
# ALERT ALERT AWOOOGA - Heinz Brandenburg
# https://twitter.com/HzBrandenburg/status/1258414163833094154

# (1) check this for specific wave7+/wave6+ respondent split
# (2) think about comparing datasets with datasets on matching variables!

## FIXED IN BIG BES UPDATE!

In [278]:
td = (pd.to_datetime("7 May 2015") - pd.to_datetime("28 Feb 2014") )
td.days/365.25

1.1854893908281998

In [15]:
search(BES_Panel,"ukCitizen")
# ukCitizen, euCitizen, commonwealthCitizen, otherCitizen

# ... misses "Irish" citizens who have access

ukCitizenW1     30575
ukCitizenW8      6338
ukCitizenW14    31063
ukCitizenW15    30842
ukCitizenW16    37959
ukCitizenW17     3564
dtype: int64

In [32]:
commonwealthCitizen = BES_Panel[search(BES_Panel,"commonwealthCitizen").index].fillna(method='ffill',axis=1)[search(BES_Panel,"commonwealthCitizen").index[-1]]

In [16]:
ukCitizen = BES_Panel[search(BES_Panel,"ukCitizen").index].fillna(method='ffill',axis=1)[search(BES_Panel,"ukCitizen").index[-1]]

In [31]:
(ukCitizen=="Yes").sum(),ukCitizen.notnull().sum()

(77413, 79532)

In [33]:
(commonwealthCitizen=="Yes").sum(),commonwealthCitizen.notnull().sum()

(2776, 79532)

In [35]:
((ukCitizen=="Yes")|(commonwealthCitizen=="Yes")).sum()

77836

In [36]:
79532-77836

1696

In [ ]:
ukCitizen = BES_Panel[search(BES_Panel,"registeredW\d+").index].fillna(method='ffill',axis=1)[search(BES_Panel,"registeredW\d+").index[-1]]

In [25]:
search(BES_Panel,"registeredW\d+")

registeredW2     25814
registeredW3     27819
registeredW4     31532
registeredW6     30258
registeredW7     30895
registeredW8     33502
registeredW11    30956
registeredW13    31136
registeredW17    34366
registeredW19    32177
dtype: int64

In [37]:
(BES_Panel[search(BES_Panel,"registeredW\d+").index]=="Don't know").sum()

registeredW2     581
registeredW3     684
registeredW4     644
registeredW6     323
registeredW7     503
registeredW8     416
registeredW11    473
registeredW13    279
registeredW17    588
registeredW19    259
dtype: int64

In [28]:
(BES_Panel[search(BES_Panel,"registeredW\d+").index]=="No").sum()

registeredW2     396
registeredW3     360
registeredW4     406
registeredW6     333
registeredW7     249
registeredW8     278
registeredW11    326
registeredW13    219
registeredW17    260
registeredW19    182
dtype: int64

In [23]:
(BES_Panel[search(BES_Panel,"reasonNotRegistered").index]=="Yes").sum()

reasonNotRegistered_1W2        46
reasonNotRegistered_2W2        32
reasonNotRegistered_3W2        25
reasonNotRegistered_4W2        48
reasonNotRegistered_5W2        30
reasonNotRegistered_6W2        40
reasonNotRegistered_7W2       109
reasonNotRegistered_8W2        26
reasonNotRegistered_noneW2      0
reasonNotRegistered_dkW2       32
reasonNotRegistered_1W3        51
reasonNotRegistered_2W3         9
reasonNotRegistered_3W3        29
reasonNotRegistered_4W3        43
reasonNotRegistered_5W3        39
reasonNotRegistered_6W3        24
reasonNotRegistered_7W3       116
reasonNotRegistered_8W3        22
reasonNotRegistered_noneW3      0
reasonNotRegistered_dkW3       26
reasonNotRegistered_1W4       119
reasonNotRegistered_2W4        12
reasonNotRegistered_3W4        26
reasonNotRegistered_4W4        47
reasonNotRegistered_5W4        48
reasonNotRegistered_6W4        29
reasonNotRegistered_7W4        77
reasonNotRegistered_8W4        31
reasonNotRegistered_noneW4      0
reasonNotRegis

In [44]:
df

turnoutUKGeneralW1__Very likely that I will vote  \
1607                                                NaN   
1644                                                0.0   
1709                                                NaN   
2063                                                0.0   
2072                                                4.0   
...                                                 ...   
96062                                               NaN   
96098                                               NaN   
96186                                               NaN   
96681                                               4.0   
97021                                               NaN   

       turnoutUKGeneralW2__Very likely that I will vote  \
1607                                                NaN   
1644                                                0.0   
1709                                                NaN   
2063                                                0.0   
2072                                                4.0   
...                                                 ...   
96062                                               NaN   
96098                                               NaN   
96186                                               NaN   
96681                                               3.0   
97021                                               NaN   

       turnoutUKGeneralW3__Very likely that I will vote  \
1607                                                0.0   
1644                                                0.0   
1709                                                NaN   
2063                                                NaN   
2072                                                4.0   
...                                                 ...   
96062                                               4.0   
96098                                               4.0   
96186                                               NaN   
96681                                               4.0   
97021                                               NaN   

       turnoutUKGeneralW4__Very likely that I will vote  \
1607                                                0.0   
1644                                                0.0   
1709                                                3.0   
2063                                                0.0   
2072                                                4.0   
...                                                 ...   
96062                                               4.0   
96098                                               3.0   
96186                                               4.0   
96681                                               3.0   
97021                                               0.0   

       turnoutUKGeneralW5__Very likely that I will vote  \
1607                                                0.0   
1644                                                0.0   
1709                                                3.0   
2063                                                0.0   
2072                                                4.0   
...                                                 ...   
96062                                               0.0   
96098                                               3.0   
96186                                               0.0   
96681                                               4.0   
97021                                               0.0   

       turnoutUKGeneralW7__Very likely that I will vote  \
1607                                                0.0   
1644                                                NaN   
1709                                                2.0   
2063                                                0.0   
2072                                                4.0   
...                                                 ...   
96062                                            

In [46]:
%%time
pattern = "reasonNotRegistered_1W6_Yes"
name = "reasonNotRegistered_1W6_Yes"
corr_type = "spearman"
mask = BES_Panel["wave6"] & (BES_Panel["registeredW6"]=="No")
df = BES_reduced_with_na[mask].astype('float32')
corr_df,corr_type = make_corr_summary(df, name, corr_type, pattern=pattern,min_sample_size=10)
display_corr(corr_df, name, corr_type, text_wrap_length=50)

Top 20 positive (spearman) correlations for reasonNotRegistered_1W6_Yes r p N 
 
 partyMemberPastW10_Plaid Cymru 
 1.000000 
 0.00 
 10 
 
 
 freeSpeechMilitaryTeachW15__Definitely 
 0.990000 
 0.00 
 13 
 
 
 reasonNotRegistered_1W8_Yes 
 0.950000 
 0.00 
 36 
 
 
 noDealEmotions_5W15_Yes 
 0.820000 
 0.00 
 13 
 
 
 euID5W7__Strongly agree 
 0.760000 
 0.01 
 10 
 
 
 labHopeW14_Yes 
 0.760000 
 0.01 
 10 
 
 
 profile_work_statW14_Full time student 
 0.740000 
 0.00 
 63 
 
 
 warmMuslimW11__Very warm 
 0.730000 
 0.00 
 13 
 
 
 freeSpeechIslamistSpeechW15__Definitely 
 0.720000 
 0.01 
 13 
 
 
 workingStatusW14W15_Full time university student 
 0.720000 
 0.00 
 71 
 
 
 childvalues_9W14_Yes 
 0.710000 
 0.01 
 12 
 
 
 likeDaveyW19__Strongly like 
 0.690000 
 0.00 
 17 
 
 
 noDealEmotions_7W15_Yes 
 0.690000 
 0.01 
 13 
 
 
 partyIdSqueezeW14_Liberal Democrat 
 0.680000 
 0.00 
 27 
 
 
 likeWoodW6__Strongly like 
 0.670000 
 0.01 
 13 
 
 
 workingStatusW6_W12_Other full time student 
 0.620000 
 0.00 
 253 
 
 
 selfOccSuperviseW16_Yes 
 0.600000 
 0.01 
 19 
 
 
 ptvGrnW12__Very likely 
 0.600000 
 0.01 
 17 
 
 
 subjClassW17_Yes, middle class 
 0.590000 
 0.00 
 55 
 
 
 passportOtherW9_Yes 
 0.590000 
 0.00 
 93 
 
 Top 20 negative (spearman) correlations for reasonNotRegistered_1W6_Yes r p N 
 
 ukCitizenW1_Yes 
 -0.710000 
 0.00 
 125 
 
 
 reasonNotRegistered_4W7_Yes 
 -0.690000 
 0.00 
 31 
 
 
 partyContactConW18_Yes 
 -0.670000 
 0.00 
 16 
 
 
 occCheckW17__Very well 
 -0.620000 
 0.01 
 18 
 
 
 britishnessW2__Very strongly British 
 -0.610000 
 0.00 
 125 
 
 
 partyContactLabW12_Yes 
 -0.590000 
 0.01 
 19 
 
 
 ageW15 
 -0.570000 
 0.00 
 58 
 
 
 discrimWhiteW10__A lot of discrimination against 
 -0.560000 
 0.00 
 80 
 
 
 ageGroup__66+ 
 -0.550000 
 0.00 
 260 
 
 
 Age 
 -0.550000 
 0.00 
 260 
 
 
 registeredW13_Yes - at $postcode_hidden 
 -0.530000 
 0.00 
 78 
 
 
 motherCountryBornW8_UK 
 -0.530000 
 0.00 
 103 
 
 
 scotRefTurnoutW3_W14_No, I did not vote 
 -0.520000 
 0.00 
 38 
 
 
 countryOfBirthW17_England 
 -0.510000 
 0.00 
 60 
 
 
 selfEUCertainW4__Very certain 
 -0.500000 
 0.01 
 25 
 
 
 englishnessW2__Very strongly English 
 -0.500000 
 0.00 
 125 
 
 
 dutyLongW11_Choice 
 -0.490000 
 0.00 
 32 
 
 
 effectsEUTradeW15__Much higher 
 -0.490000 
 0.00 
 48 
 
 
 antiIntellectualW17__Strongly agree 
 -0.490000 
 0.00 
 52 
 
 
 fatherCountryBornW8_UK 
 -0.480000 
 0.00 
 103

Wall time: 50.5 s


In [47]:
search(BES_Panel,"countryOfBirth")

countryOfBirth       36842
countryOfBirthW17    34366
dtype: int64

In [49]:
BES_Panel["countryOfBirth"].value_counts()

England                                 25332
Scotland                                 6011
Wales                                    2834
Other: European Union member country      704
Other: Rest of world                      703
Other: Commonwealth member country        601
Prefer not to answer                      272
Republic of Ireland                       199
Northern Ireland                          186
Name: countryOfBirth, dtype: int64

In [40]:
search(BES_reduced_with_na, "reasonNotRegistered_1W6")
# BES_Panel["reasonNotRegistered_1W6"]

reasonNotRegistered_1W6_Yes    260
dtype: int64

In [42]:
df

turnoutUKGeneralW1__Very likely that I will vote  \
1607                                                NaN   
1644                                                0.0   
1709                                                NaN   
2063                                                0.0   
2072                                                4.0   
...                                                 ...   
96062                                               NaN   
96098                                               NaN   
96186                                               NaN   
96681                                               4.0   
97021                                               NaN   

       turnoutUKGeneralW2__Very likely that I will vote  \
1607                                                NaN   
1644                                                0.0   
1709                                                NaN   
2063                                                0.0   
2072                                                4.0   
...                                                 ...   
96062                                               NaN   
96098                                               NaN   
96186                                               NaN   
96681                                               3.0   
97021                                               NaN   

       turnoutUKGeneralW3__Very likely that I will vote  \
1607                                                0.0   
1644                                                0.0   
1709                                                NaN   
2063                                                NaN   
2072                                                4.0   
...                                                 ...   
96062                                               4.0   
96098                                               4.0   
96186                                               NaN   
96681                                               4.0   
97021                                               NaN   

       turnoutUKGeneralW4__Very likely that I will vote  \
1607                                                0.0   
1644                                                0.0   
1709                                                3.0   
2063                                                0.0   
2072                                                4.0   
...                                                 ...   
96062                                               4.0   
96098                                               3.0   
96186                                               4.0   
96681                                               3.0   
97021                                               0.0   

       turnoutUKGeneralW5__Very likely that I will vote  \
1607                                                0.0   
1644                                                0.0   
1709                                                3.0   
2063                                                0.0   
2072                                                4.0   
...                                                 ...   
96062                                               0.0   
96098                                               3.0   
96186                                               0.0   
96681                                               4.0   
97021                                               0.0   

       turnoutUKGeneralW7__Very likely that I will vote  \
1607                                                0.0   
1644                                                NaN   
1709                                                2.0   
2063                                                0.0   
2072                                                4.0   
...                                                 ...   
96062                                            

In [488]:
pan_dataset_votes["2014EEVote"+"_complete"].value_counts(dropna=False)

Unknown                                     60988
Did not vote                                 8197
United Kingdom Independence Party (UKIP)     6365
Too Young To Vote                            5258
Labour                                       5074
Conservative                                 3899
Green Party                                  2521
Scottish National Party (SNP)                1443
Liberal Democrat                             1379
Not Present                                   629
Other                                         565
Plaid Cymru                                   437
Don't know                                    308
British National Party (BNP)                   95
Name: 2014EEVote_complete, dtype: int64

In [489]:
pan_dataset_votes["2019EEVote"+"_complete"].value_counts(dropna=False)

Unknown                                     58536
Did not vote                                12034
Brexit Party                                 9229
Liberal Democrat                             5369
Green Party                                  3517
Labour                                       2510
Conservative                                 1715
Scottish National Party (SNP)                 945
Change UK- The Independent Group              882
Not Present                                   629
Other                                         526
United Kingdom Independence Party (UKIP)      399
Don't know                                    379
Plaid Cymru                                   294
Too Young To Vote                             194
Name: 2019EEVote_complete, dtype: int64

In [469]:
pan_dataset_votes["2005GEVote_complete"].value_counts(dropna=False)

Unknown                                     25473
Too Young To Vote                           19789
Labour                                      15323
Conservative                                11666
Liberal Democrat                             9423
Did not vote                                 4783
Scottish National Party                      3792
Other                                        1770
Don't know                                   1692
United Kingdom Independence Party (UKIP)      684
Not Present                                   629
Plaid Cymru                                   625
Green Party                                   614
British National Party (BNP)                  505
Ambiguous category                            283
Respect                                       104
Veritas                                         3
Name: 2005GEVote_complete, dtype: int64

In [470]:
pan_dataset_votes["2010GEVote_complete"].value_counts(dropna=False)

Unknown                                     23666
Labour                                      16448
Conservative                                14808
Liberal Democrat                            12653
Too Young To Vote                           11716
Scottish National Party                      6067
Did not vote                                 4436
Other                                        1376
United Kingdom Independence Party (UKIP)     1211
Don't know                                    925
British National Party (BNP)                  841
Plaid Cymru                                   800
Ambiguous category                            728
Green Party                                   663
Not Present                                   629
Respect                                       191
Name: 2010GEVote_complete, dtype: int64

In [471]:
pan_dataset_votes["2015GEVote_complete"].value_counts(dropna=False)

Conservative                                23223
Labour                                      21820
Unknown                                     12616
Did not vote                                10971
United Kingdom Independence Party (UKIP)     7424
Liberal Democrat                             5950
Green Party                                  3817
Scottish National Party (SNP)                3813
Too Young To Vote                            3005
Don't know                                   2206
Other                                        1084
Not Present                                   629
Plaid Cymru                                   535
British National Party (BNP)                   65
Name: 2015GEVote_complete, dtype: int64

In [472]:
pan_dataset_votes["2017GEVote_complete"].value_counts(dropna=False)

Unknown                                     26009
Conservative                                23716
Labour                                      23624
Did not vote                                 8975
Liberal Democrat                             5245
Scottish National Party (SNP)                2515
United Kingdom Independence Party (UKIP)     1482
Too Young To Vote                            1403
Green Party                                  1274
Don't know                                   1173
Other                                         716
Not Present                                   629
Plaid Cymru                                   396
British National Party (BNP)                    1
Name: 2017GEVote_complete, dtype: int64

In [473]:
pan_dataset_votes["2019GEVote"].value_counts(dropna=False)

NaN                                         65715
Conservative                                12666
Labour                                       8244
Liberal Democrat                             3579
Did not vote                                 3159
Scottish National Party (SNP)                1249
Green Party                                   858
Brexit Party                                  722
Other                                         553
Don't know                                    231
Plaid Cymru                                   182
United Kingdom Independence Party (UKIP)        0
Too Young To Vote                               0
Name: 2019GEVote, dtype: int64

In [474]:
pan_dataset_votes["2019GEVote_complete"].value_counts(dropna=False)

Unknown                                     64352
Conservative                                12890
Labour                                       8453
Liberal Democrat                             3648
Did not vote                                 3159
Scottish National Party (SNP)                1281
Green Party                                   894
Brexit Party                                  746
Not Present                                   629
Other                                         603
Don't know                                    317
Plaid Cymru                                   183
United Kingdom Independence Party (UKIP)        3
Too Young To Vote                               0
Name: 2019GEVote_complete, dtype: int64

In [490]:
pan_dataset_votes.stack().unique()

array(['Plaid Cymru', 'British National Party (BNP)', 'Labour',
       'Brexit Party', 'Leave the EU',
       'United Kingdom Independence Party (UKIP)', 'Conservative',
       'Stay/remain in the EU', 'Did not vote', 'Unknown', "Don't know",
       'Too Young To Vote', 'Liberal Democrat', 'Scottish National Party',
       'Other', 'Green Party', 'Ambiguous category',
       'Scottish National Party (SNP)', 'Respect',
       'Change UK- The Independent Group', 'Not Present', 'Veritas'],
      dtype=object)

In [491]:
pan_dataset_votes.to_pickle(BES_small_data_files + "pan_dataset_votes"+".zip", compression='zip')

In [383]:
search(BES_Panel,"euroTurnout")

euroTurnoutW1          30542
euroTurnoutRetroW2     30262
euroTurnoutRetroW16    37959
dtype: int64

In [381]:
search(BES_Panel,"euroElection")

euroElectionVoteW1             30537
euroElectionVoteW2             22188
euroElectionVoteW16            25837
euroElectionVoteNonVoterW16    12122
dtype: int64

In [379]:
search(BES_Panel,"pastvote")

euRefpastVoteW11        27999
euRefpastVoteW12        31173
euRefpastVoteW13        28288
euRefpastVoteW14         3822
euRefpastVoteW15        27007
euRefpastVoteW17         2643
pastvote_EURefW17       34352
pastvote_eup_2019W18    27133
pastvote_ge_2019W19     31443
pastvote_eup_2019W19    23186
dtype: int64

In [306]:
search(BES_Panel,"2010")

profile_past_vote_2010    65581
dtype: int64

In [340]:
corrs = pd.get_dummies(BES_Panel[["profile_past_vote_2005","profile_past_vote_2010","profile_past_vote_2015"]]).corr()
corrs[[x for x in corrs.columns if "profile_past_vote_2010" in x]].idxmax(axis=1)

profile_past_vote_2005_I would not vote                                      profile_past_vote_2010_I would not vote
profile_past_vote_2005_Conservative                                              profile_past_vote_2010_Conservative
profile_past_vote_2005_Labour                                                          profile_past_vote_2010_Labour
profile_past_vote_2005_Liberal Democrat                                      profile_past_vote_2010_Liberal Democrat
profile_past_vote_2005_Scottish National Party (SNP)               profile_past_vote_2010_Scottish National Party...
profile_past_vote_2005_Plaid Cymru                                                profile_past_vote_2010_Plaid Cymru
profile_past_vote_2005_United Kingdom Independence Party (UKIP)    profile_past_vote_2010_United Kingdom Independ...
profile_past_vote_2005_Green Party                                                profile_past_vote_2010_Green Party
profile_past_vote_2005_British National Party (BNP)             

In [359]:
list(search(BES_reduced_with_na,"ptv.*W1_").index)

['ptvConW1__Very likely',
 'ptvLabW1__Very likely',
 'ptvLDW1__Very likely',
 'ptvSNPW1__Very likely',
 'ptvPCW1__Very likely',
 'ptvUKIPW1__Very likely',
 'ptvGrnW1__Very likely',
 'ptvBNPW1__Very likely']

In [361]:
corrs['ptvConW1__Very likely', 'ptvLabW1__Very likely',

ptvConW1__Very likely                                              1.000000
ptvLabW1__Very likely                                             -0.439090
ptvLDW1__Very likely                                               0.132162
ptvSNPW1__Very likely                                             -0.360742
ptvPCW1__Very likely                                              -0.167955
ptvUKIPW1__Very likely                                             0.235519
ptvGrnW1__Very likely                                             -0.215325
ptvBNPW1__Very likely                                              0.053291
profile_past_vote_2010_I would not vote                           -0.099291
profile_past_vote_2010_Conservative                                0.650714
profile_past_vote_2010_Labour                                     -0.351950
profile_past_vote_2010_Liberal Democrat                           -0.157116
profile_past_vote_2010_Scottish National Party (SNP)              -0.135695
profile_past

In [362]:
# pd.get_dummies(BES_Panel["profile_past_vote_2010"])
corrs= pd.concat([BES_reduced_with_na[search(BES_reduced_with_na,"ptv.*W1_").index].astype('float32'),
           pd.get_dummies(BES_Panel[["profile_past_vote_2010"]])],axis=1).corr()
# corrs
corrs[list(search(BES_reduced_with_na,"ptv.*W1_").index)].idxmax(axis=1)

ptvConW1__Very likely                                               ptvConW1__Very likely
ptvLabW1__Very likely                                               ptvLabW1__Very likely
ptvLDW1__Very likely                                                 ptvLDW1__Very likely
ptvSNPW1__Very likely                                               ptvSNPW1__Very likely
ptvPCW1__Very likely                                                 ptvPCW1__Very likely
ptvUKIPW1__Very likely                                             ptvUKIPW1__Very likely
ptvGrnW1__Very likely                                               ptvGrnW1__Very likely
ptvBNPW1__Very likely                                               ptvBNPW1__Very likely
profile_past_vote_2010_I would not vote                             ptvBNPW1__Very likely
profile_past_vote_2010_Conservative                                 ptvConW1__Very likely
profile_past_vote_2010_Labour                                       ptvLabW1__Very likely
profile_pa

In [363]:
# pd.get_dummies(BES_Panel["profile_past_vote_2010"])
corrs= pd.concat([BES_reduced_with_na[search(BES_reduced_with_na,"ptv.*W1_").index].astype('float32'),
           pd.get_dummies(BES_Panel[["profile_past_vote_2005"]])],axis=1).corr()
# corrs
corrs[list(search(BES_reduced_with_na,"ptv.*W1_").index)].idxmax(axis=1)

ptvConW1__Very likely                                               ptvConW1__Very likely
ptvLabW1__Very likely                                               ptvLabW1__Very likely
ptvLDW1__Very likely                                                 ptvLDW1__Very likely
ptvSNPW1__Very likely                                               ptvSNPW1__Very likely
ptvPCW1__Very likely                                                 ptvPCW1__Very likely
ptvUKIPW1__Very likely                                             ptvUKIPW1__Very likely
ptvGrnW1__Very likely                                               ptvGrnW1__Very likely
ptvBNPW1__Very likely                                               ptvBNPW1__Very likely
profile_past_vote_2005_I would not vote                             ptvBNPW1__Very likely
profile_past_vote_2005_Conservative                                 ptvConW1__Very likely
profile_past_vote_2005_Labour                                       ptvLabW1__Very likely
profile_pa

In [364]:
# pd.get_dummies(BES_Panel["profile_past_vote_2010"])
corrs= pd.concat([BES_reduced_with_na[search(BES_reduced_with_na,"ptv.*W5_").index].astype('float32'),
           pd.get_dummies(BES_Panel[["profile_past_vote_2015"]])],axis=1).corr()
# corrs
corrs[list(search(BES_reduced_with_na,"ptv.*W5_").index)].idxmax(axis=1)

ptvConW5__Very likely                                               ptvConW5__Very likely
ptvLabW5__Very likely                                               ptvLabW5__Very likely
ptvLDW5__Very likely                                                 ptvLDW5__Very likely
ptvSNPW5__Very likely                                               ptvSNPW5__Very likely
ptvPCW5__Very likely                                                 ptvPCW5__Very likely
ptvUKIPW5__Very likely                                             ptvUKIPW5__Very likely
ptvGrnW5__Very likely                                               ptvGrnW5__Very likely
ptvBNPW5__Very likely                                               ptvBNPW5__Very likely
profile_past_vote_2015_Conservative                                 ptvConW5__Very likely
profile_past_vote_2015_Labour                                       ptvLabW5__Very likely
profile_past_vote_2015_Liberal Democrat                              ptvLDW5__Very likely
profile_pa

In [ ]:
corrs = pd.get_dummies(BES_Panel[["profile_past_vote_2005","profile_past_vote_2010","profile_past_vote_2015",]]).corr()
corrs[[x for x in corrs.columns if "profile_past_vote_2010" in x]].idxmax(axis=1)

In [365]:
BES_Panel["profile_past_vote_2010"].value_counts()

Conservative                                17934
Labour                                      16643
Liberal Democrat                            12853
Scottish National Party (SNP)                6158
I would not vote                             5001
United Kingdom Independence Party (UKIP)     1220
Brexit Party                                 1126
Don't know                                    952
British National Party (BNP)                  868
Plaid Cymru                                   829
10.0                                          741
Green Party                                   686
Other                                         371
Change UK- The Independent Group              199
Name: profile_past_vote_2010, dtype: int64

In [366]:
BES_Panel["profile_past_vote_2005"].value_counts()
Change UK- The Independent Group                3 -> veritas?
Brexit Party                                  116 -> respect

Conservative                                16407
Labour                                      15435
Liberal Democrat                             9582
I would not vote                             4791
Scottish National Party (SNP)                3857
Don't know                                   1694
An independent candidate                     1611
United Kingdom Independence Party (UKIP)      691
Plaid Cymru                                   647
Green Party                                   622
British National Party (BNP)                  523
Other                                         357
10.0                                          295
Brexit Party                                  116
Change UK- The Independent Group                3
Name: profile_past_vote_2005, dtype: int64

In [368]:
BES_Panel["profile_past_vote_2010"].cat.categories

Index([                        'I would not vote',
                                   'Conservative',
                                         'Labour',
                               'Liberal Democrat',
                  'Scottish National Party (SNP)',
                                    'Plaid Cymru',
       'United Kingdom Independence Party (UKIP)',
                                    'Green Party',
                   'British National Party (BNP)',
                                          'Other',
                                             10.0,
               'Change UK- The Independent Group',
                                   'Brexit Party',
                                     'Don't know'],
      dtype='object')

In [367]:
BES_Panel["profile_past_vote_2005"].cat.categories

Index([                        'I would not vote',
                                   'Conservative',
                                         'Labour',
                               'Liberal Democrat',
                  'Scottish National Party (SNP)',
                                    'Plaid Cymru',
       'United Kingdom Independence Party (UKIP)',
                                    'Green Party',
                   'British National Party (BNP)',
                                          'Other',
                                             10.0,
               'Change UK- The Independent Group',
                                   'Brexit Party',
                       'An independent candidate',
                                     'Don't know'],
      dtype='object')

In [370]:
BES_Panel["profile_past_vote_2005"].cat.categories
replace_dict = {'I would not vote':'Did not vote',
                'Conservative':'Conservative Party',
                'Labour':'Labour Party',
                'Liberal Democrat':'Liberal Democrats',
                'Scottish National Party (SNP)':'Scottish National Party',
                'Plaid Cymru':'Plaid Cymru',
                'United Kingdom Independence Party (UKIP)':'United Kingdom Independence Party (UKIP)',
                'Green Party':'Green Party',
                'British National Party (BNP)':'British National Party (BNP)',
                'Other': 'Some other party',
                10.0:'Ambiguous category',
                'Change UK- The Independent Group':'Veritas',
                'Brexit Party':'Respect',
                'An independent candidate':'An independent candidate',
                "Don't know":"Don't know",
               }
BES_Panel["profile_past_vote_2005"].replace(replace_dict)

Conservative Party                          16407
Labour Party                                15435
Liberal Democrats                            9582
Did not vote                                 4791
Scottish National Party                      3857
Don't know                                   1694
An independent candidate                     1611
United Kingdom Independence Party (UKIP)      691
Plaid Cymru                                   647
Green Party                                   622
British National Party (BNP)                  523
Some other party                              357
Ambiguous category                            295
Respect                                       116
Veritas                                         3
Name: profile_past_vote_2005, dtype: int64

In [310]:
BES_Panel["profile_past_vote_2010"].cat.categories
replace_dict = {'I would not vote':'Did not vote',
                'Conservative':'Conservative Party',
                'Labour':'Labour Party',
                'Liberal Democrat':'Liberal Democrats',
                'Scottish National Party (SNP)':'Scottish National Party',
                'Plaid Cymru':'Plaid Cymru',
                'United Kingdom Independence Party (UKIP)':'United Kingdom Independence Party (UKIP)',
                'Green Party':'Green Party',
                'British National Party (BNP)':'British National Party (BNP)',
                'Other': 'Some other party',
                10.0:'Ambiguous category',
                'Change UK- The Independent Group':'Respect',
                'Brexit Party':'An independent candidate',
                "Don't know":"Don't know",
               }
BES_Panel["profile_past_vote_2010"].replace(replace_dict)

Index([                        'I would not vote',
                                   'Conservative',
                                         'Labour',
                               'Liberal Democrat',
                  'Scottish National Party (SNP)',
                                    'Plaid Cymru',
       'United Kingdom Independence Party (UKIP)',
                                    'Green Party',
                   'British National Party (BNP)',
                                          'Other',
                                             10.0,
               'Change UK- The Independent Group',
                                   'Brexit Party',
                                     'Don't know'],
      dtype='object')

In [315]:
search(BES_reduced_with_na,"profile_past_vote_2010")

profile_past_vote_2010_Conservative                                59257
profile_past_vote_2010_Labour                                      59257
profile_past_vote_2010_Liberal Democrat                            59257
profile_past_vote_2010_Scottish National Party (SNP)               59257
profile_past_vote_2010_United Kingdom Independence Party (UKIP)    59257
profile_past_vote_2010_Brexit Party                                59257
profile_past_vote_2010_British National Party (BNP)                59257
profile_past_vote_2010_Plaid Cymru                                 59257
profile_past_vote_2010_10.0                                        59257
profile_past_vote_2010_Green Party                                 59257
profile_past_vote_2010_Change UK- The Independent Group            59257
dtype: int64

In [330]:
BES_reduced_with_na[search(BES_reduced_with_na,"ptv.*W1__").index].astype('float32').corrwith((BES_Panel["profile_past_vote_2010"]=="Other"))

ptvConW1__Very likely    -0.027092
ptvLabW1__Very likely    -0.031455
ptvLDW1__Very likely     -0.026559
ptvSNPW1__Very likely    -0.004223
ptvPCW1__Very likely     -0.001329
ptvUKIPW1__Very likely    0.010653
ptvGrnW1__Very likely     0.004426
ptvBNPW1__Very likely     0.005244
dtype: float64

In [ ]:
BES_Panel[ BES_Panel["profile_past_vote_2010"]=="Change UK- The Independent Group" ][search(BES_Panel,"wave\d+").index].sum()

In [325]:
BES_Panel[ BES_Panel["profile_past_vote_2010"]=="Brexit Party" ][search(BES_Panel,"wave\d+").index].sum()

wave1      10
wave2       9
wave3      14
wave4      16
wave5      17
wave6      13
wave7     225
wave8     388
wave9     325
wave10    500
wave11    582
wave12    730
wave13    638
wave14    617
wave15    494
wave16    463
wave17    406
wave18    419
wave19    354
dtype: int64

In [331]:
BES_Panel["profile_past_vote_2010"].value_counts().sort_index()

I would not vote                             5001
Conservative                                17934
Labour                                      16643
Liberal Democrat                            12853
Scottish National Party (SNP)                6158
Plaid Cymru                                   829
United Kingdom Independence Party (UKIP)     1220
Green Party                                   686
British National Party (BNP)                  868
Other                                         371
10.0                                          741
Change UK- The Independent Group              199
Brexit Party                                 1126
Don't know                                    952
Name: profile_past_vote_2010, dtype: int64

In [332]:
BES_Panel["profile_past_vote_2015"].value_counts().sort_index()

Conservative                                23284
Labour                                      21943
Liberal Democrat                             5977
Scottish National Party (SNP)                3825
Plaid Cymru                                   535
United Kingdom Independence Party (UKIP)     7434
Green Party                                  3835
British National Party (BNP)                   66
Other                                        1087
Don't know                                   2591
Did not vote                                12491
Name: profile_past_vote_2015, dtype: int64

In [313]:
BES_Panel["profile_past_vote_2015"].value_counts()

Conservative                                23284
Labour                                      21943
Did not vote                                12491
United Kingdom Independence Party (UKIP)     7434
Liberal Democrat                             5977
Green Party                                  3835
Scottish National Party (SNP)                3825
Don't know                                   2591
Other                                        1087
Plaid Cymru                                   535
British National Party (BNP)                   66
Name: profile_past_vote_2015, dtype: int64

In [314]:
BES_Panel["profile_past_vote_2005"].value_counts()

Conservative                                16407
Labour                                      15435
Liberal Democrat                             9582
I would not vote                             4791
Scottish National Party (SNP)                3857
Don't know                                   1694
An independent candidate                     1611
United Kingdom Independence Party (UKIP)      691
Plaid Cymru                                   647
Green Party                                   622
British National Party (BNP)                  523
Other                                         357
10.0                                          295
Brexit Party                                  116
Change UK- The Independent Group                3
Name: profile_past_vote_2005, dtype: int64

In [268]:
pd.crosstab(BES_Panel["euRefTurnoutW8"], BES_Panel["profile_eurefturnout"])

profile_eurefturnout             No, did not vote  Yes, voted  Don't know
euRefTurnoutW8                                                           
Very unlikely that I would vote               599         151          12
Fairly unlikely                               176         149           8
Neither likely nor unlikely                   220         326          22
Fairly likely                                 251        1594          20
Very likely that I would vote                 369       23143          43
Don't know                                    168         257          56

In [255]:
search(BES_Panel,"euref",mask = BES_Panel["profile_eurefvote"].isnull()).sort_values().tail(50)

euRefpastVoteW14                     0
euRefDoOverW15                       0
euRefVoteW15                         0
goodConductEURefW15                  0
euRefTurnoutRetroW15                 0
euRefpastVoteW15                     0
euRefTurnoutRetroW17                 0
euRefpastVoteW17                     0
euRefpastVoteW13                     0
euRefDoOverW10                       0
profile_eurefturnout                 0
euRefFinalPostW9                     0
euRefVoteW9                          0
euRefVoteHypotheticalW9              0
euRefTurnoutRetroW9                  0
goodConductEURefW9                   0
euRefVoteW16                         5
euRefDoOverW16                       5
EUrefsatisfactionControl_1W19       28
euRefVotePostW8                    207
euRefVoteSqueezeW8                 225
euRefVoteW19                       271
euRefDoOverW19                     271
pastvote_EURefW17                  282
euRefVoteW17                       283
euRefDoOverW17           

In [256]:
BES_Panel["pastvote_EURefW17"][BES_Panel["profile_eurefvote"].isnull()].value_counts()

Male      135
Female    124
3.0        19
4.0         4
Name: pastvote_EURefW17, dtype: int64

In [258]:
BES_Panel["profile_eurefvote"].value_counts()

Stay/remain in the EU    36286
Leave the EU             31721
Did not vote             10997
Don't know                 507
Name: profile_eurefvote, dtype: int64

In [263]:
BES_Panel["pastvote_EURefW17"].value_counts()
replace_dict = {"Male":'Stay/remain in the EU',"Female":'Leave the EU',3.0:'Did not vote',4.0:"Don't know"}
BES_Panel["pastvote_EURefW17"].replace(replace_dict).astype('category').cat.set_categories(BES_Panel["profile_eurefvote"].cat.categories)

Stay/remain in the EU    15266
Leave the EU             14843
Did not vote              3714
Don't know                 529
Name: pastvote_EURefW17, dtype: int64

In [260]:
BES_Panel["pastvote_EURefW17"][BES_Panel["profile_eurefvote"] == "Stay/remain in the EU"].value_counts()

Male      14469
Female      164
3.0          97
4.0          57
Name: pastvote_EURefW17, dtype: int64

In [254]:
search(BES_Panel,"euref",mask = BES_Panel["profile_eurefturnout"].isnull()).sort_values().tail(50)

euRefpastVoteW14                     0
euRefDoOverW15                       0
euRefVoteW15                         0
goodConductEURefW15                  0
euRefTurnoutRetroW15                 0
euRefpastVoteW15                     0
euRefTurnoutRetroW17                 0
euRefpastVoteW17                     0
euRefpastVoteW13                     0
euRefDoOverW10                       0
profile_eurefturnout                 0
euRefFinalPostW9                     0
euRefVoteW9                          0
euRefVoteHypotheticalW9              0
euRefTurnoutRetroW9                  0
goodConductEURefW9                   0
euRefVoteW16                         5
euRefDoOverW16                       5
EUrefsatisfactionControl_1W19       28
euRefVotePostW8                    207
euRefVoteSqueezeW8                 225
euRefVoteW19                       271
euRefDoOverW19                     271
pastvote_EURefW17                  282
euRefVoteW17                       283
euRefDoOverW17           

In [ ]:
pastvote_EURefW17

In [248]:
pan_dataset_votes["2017GEVote"+"_complete"].value_counts(dropna=False)

Unknown                                     26037
Labour                                      23817
Conservative                                23760
Did not vote                                10041
Liberal Democrat                             5275
Scottish National Party (SNP)                2531
United Kingdom Independence Party (UKIP)     1484
Green Party                                  1285
Don't know                                   1179
Other                                         723
Not Present                                   629
Plaid Cymru                                   396
British National Party (BNP)                    1
Name: 2017GEVote_complete, dtype: int64

In [244]:
pan_dataset_votes[new_vote_var+"_complete"][too_young_to_vote].value_counts()

Did not vote                                666
Labour                                       29
Conservative                                  7
Liberal Democrat                              4
Green Party                                   2
Scottish National Party (SNP)                 2
Don't know                                    1
Other                                         1
Not Present                                   0
British National Party (BNP)                  0
United Kingdom Independence Party (UKIP)      0
Plaid Cymru                                   0
Name: 2017GEVote_complete, dtype: int64

In [210]:
BES_Panel["profile_past_vote_2017"][BES_Panel["generalElectionVoteRetro2017W18"].notnull()].value_counts()


Conservative                                3422
Labour                                      2803
Liberal Democrat                             650
Scottish National Party (SNP)                316
Did not vote                                 289
United Kingdom Independence Party (UKIP)     199
Green Party                                  154
Don't know                                    98
Other                                         87
Plaid Cymru                                   45
British National Party (BNP)                   0
Name: profile_past_vote_2017, dtype: int64

In [211]:
BES_Panel["generalElectionVoteRetro2017W18"].value_counts()

Conservative                                3478
Labour                                      2409
Liberal Democrat                             793
United Kingdom Independence Party (UKIP)     416
Scottish National Party (SNP)                336
Don't know                                   236
Green Party                                  224
Other                                        119
Plaid Cymru                                   52
Name: generalElectionVoteRetro2017W18, dtype: int64

In [212]:
BES_Panel["generalElectionVoteRetro2017W18"][BES_Panel["profile_past_vote_2017"]=="Did not vote"].value_counts()


Conservative                                106
Labour                                       73
Liberal Democrat                             38
United Kingdom Independence Party (UKIP)     34
Scottish National Party (SNP)                17
Don't know                                   11
Other                                         5
Green Party                                   4
Plaid Cymru                                   1
Name: generalElectionVoteRetro2017W18, dtype: int64

In [216]:
search(BES_Panel,"gen.*Retro")

generalElectionVoteRetro2017W18     8063
econGenRetroW1                     30549
econGenRetroW2                     30260
econGenRetroW3                     27829
econGenRetroW4                     31550
genElecTurnoutRetroW6              30258
econGenRetroW6                      7522
econGenRetroW7                     30895
econGenRetroW8                     15200
econGenRetroW10                    30237
econGenRetroW11                    30956
econGenRetroW12                     8984
genElecTurnoutRetroW13             31136
econGenRetroW13                    31136
econGenRetroW14                    31063
econGenRetroW15                    30842
econGenRetroW16                    37959
econGenRetroW17                    34366
genElecTurnoutRetro2017W18          9369
genElecTurnoutRetroW19             32177
dtype: int64

In [220]:
BES_Panel["generalElectionVoteRetro2017W18"].value_counts()

Conservative                                3478
Labour                                      2409
Liberal Democrat                             793
United Kingdom Independence Party (UKIP)     416
Scottish National Party (SNP)                336
Don't know                                   236
Green Party                                  224
Other                                        119
Plaid Cymru                                   52
Name: generalElectionVoteRetro2017W18, dtype: int64

In [217]:
BES_Panel["genElecTurnoutRetro2017W18"].value_counts()

Yes, voted    8177
2.0            962
Don't know     230
Name: genElecTurnoutRetro2017W18, dtype: int64

In [222]:
BES_Panel["genElecTurnoutRetro2017W18"][BES_Panel["voted2017W19"]=="No, did not vote"].value_counts()

2.0           465
Yes, voted    204
Don't know     44
Name: genElecTurnoutRetro2017W18, dtype: int64

In [223]:
BES_Panel["generalElectionVoteRetro2017W18"][BES_Panel["voted2017W19"]=="No, did not vote"].value_counts()

Conservative                                73
Labour                                      46
United Kingdom Independence Party (UKIP)    27
Liberal Democrat                            26
Scottish National Party (SNP)               11
Don't know                                   7
Green Party                                  4
Other                                        3
Plaid Cymru                                  0
Name: generalElectionVoteRetro2017W18, dtype: int64

In [ ]:
BES_Panel["genElecTurnoutRetro2017W18"].value_counts()

In [186]:
BES_Panel["generalElectionVoteRetro2017W18"].value_counts()

Conservative                                3478
Labour                                      2409
Liberal Democrat                             793
United Kingdom Independence Party (UKIP)     416
Scottish National Party (SNP)                336
Don't know                                   236
Green Party                                  224
Other                                        119
Plaid Cymru                                   52
Name: generalElectionVoteRetro2017W18, dtype: int64

In [165]:
BES_Panel["generalElectionVoteW19"][last_chunk_unassigned].astype('object').replace(replace_dict)\
        .astype('category').cat.set_categories(pan_dataset_votes[new_vote_var+"_complete"].cat.categories,ordered =True)

37                              Labour
57                              Labour
87                        Conservative
90                          Don't know
91       Scottish National Party (SNP)
                     ...              
31960                           Labour
32031                            Green
32080                           Labour
32155                           Labour
32158                     Conservative
Name: generalElectionVoteW19, Length: 734, dtype: category
Categories (11, object): [Conservative < Labour < Liberal Democrat < Scottish National Party (SNP) ... Other < Don't know < Did not vote < Not Present]

In [66]:
# %%time
# pattern = "profile_eurefturnout_Yes"
# name = "profile_eurefturnout_Yes"
# corr_type = "spearman"
# mask = BES_Panel["profile_eurefturnout"].notnull()
# corr_df,corr_type = make_corr_summary(BES_reduced_with_na[mask].astype('float32'),
#                                       name, corr_type, pattern=pattern)
# display_corr(corr_df, name, corr_type, text_wrap_length=50)

In [75]:
BES_Panel["pastvote_ge_2019W19"].value_counts().sum(), BES_Panel["wave19"].sum()

(31443, 32806)

In [141]:

pan_dataset_votes.loc[last_chunk_unassigned,new_vote_var+"_complete"].cat.categories
# = BES_Panel["generalElectionVoteW19"][last_chunk_unassigned].astype('object').replace(replace_dict).values

Index(['Conservative', 'Labour', 'Liberal Democrat',
       'Scottish National Party (SNP)', 'Plaid Cymru', 'Brexit Party', 'Green',
       'Other', 'Don't know', 'Did not vote', 'Not Present'],
      dtype='object')

In [140]:
BES_Panel["generalElectionVoteW19"][last_chunk_unassigned].astype('object').replace(replace_dict).value_counts(dropna=False)

Conservative                                224
Labour                                      209
Don't know                                   86
Liberal Democrat                             69
Other                                        50
Green                                        36
Scottish National Party (SNP)                32
Brexit Party                                 24
United Kingdom Independence Party (UKIP)      3
Plaid Cymru                                   1
Name: generalElectionVoteW19, dtype: int64

In [142]:
BES_Panel["pastvote_ge_2019W19"].value_counts()

Conservative                     12666
Labour                            8244
Liberal Democrat                  3579
Did not vote                      3159
Scottish National Party (SNP)     1249
Green                              858
Brexit Party                       722
Other                              553
Don't know                         231
Plaid Cymru                        182
Name: pastvote_ge_2019W19, dtype: int64

In [ ]:
[BES_Panel["pastvote_ge_2019W19"]=="Other"]

In [78]:
GE2019_unaccounted = (BES_Panel["wave19"] & BES_Panel["pastvote_ge_2019W19"].isnull())
# 1363 unaccounted for

In [80]:
BES_Panel[GE2019_unaccounted]["profile_turnout_2017"].value_counts()

Yes, voted          608
No, did not vote    107
Don't know           16
Name: profile_turnout_2017, dtype: int64

In [85]:
# search(BES_Panel,"wt_new")
nowtsW19 = (BES_Panel["wt_new_W19_result"].isnull() & BES_Panel["wave19"])

# bunch of people who only appear to exist in wave/id/wt_full_W1/gor/country/pcon/pano variables!
# 629

In [103]:
# BES_Panel[nowtsW19].notnull().sum().sort_values().tail(60)
# nowtsW19.sum()

In [106]:
BES_Panel["pastvote_ge_2019W19"].value_counts().sum() - BES_Panel["wt_new_W19_result"].notnull().sum()

-734

In [109]:
last_chunk_unassigned = (BES_Panel["pastvote_ge_2019W19"].isnull() & BES_Panel["wt_new_W19_result"].notnull())

In [111]:
BES_Panel[last_chunk_unassigned]["profile_turnout_2017"].value_counts()

Yes, voted          608
No, did not vote    107
Don't know           16
Name: profile_turnout_2017, dtype: int64

In [128]:
BES_Panel[last_chunk_unassigned]["generalElectionVoteW19"].value_counts()

Conservative                                224
Labour                                      209
Don't know                                   86
Liberal Democrat                             69
Other                                        45
Green Party                                  36
Scottish National Party (SNP)                32
Brexit Party                                 24
An independent candidate                      5
United Kingdom Independence Party (UKIP)      3
Plaid Cymru                                   1
Name: generalElectionVoteW19, dtype: int64

In [127]:
BES_Panel["pastvote_ge_2019W19"].cat.categories

Index(['Conservative', 'Labour', 'Liberal Democrat',
       'Scottish National Party (SNP)', 'Plaid Cymru', 'Brexit Party', 'Green',
       'Other', 'Don't know', 'Did not vote'],
      dtype='object')

In [ ]:
replace_dict = {'Green Party':'Green','An independent candidate':'Other'}

In [117]:
BES_Panel[last_chunk_unassigned]["generalElectionVoteW19"].value_counts().sum()

734

In [119]:
BES_Panel[last_chunk_unassigned]["generalElectionVoteW18"].value_counts().sum()

482

In [121]:
BES_Panel[last_chunk_unassigned]["generalElectionVoteW17"].value_counts().sum()

453

In [493]:
LD2005 = pan_dataset_votes["2005GEVote_complete"]=="Liberal Democrat"
LD2010 = pan_dataset_votes["2010GEVote_complete"]=="Liberal Democrat"
LD05_10 = LD2005|LD2010
LD05_10.sum()

16055

In [499]:
LD05_10_tot = (pan_dataset_votes["2005GEVote_complete"]!="Unknown")|(pan_dataset_votes["2010GEVote_complete"]!="Unknown")

In [502]:
LAB2015 = pan_dataset_votes["2015GEVote_complete"]=="Labour"
LAB2017 = pan_dataset_votes["2017GEVote_complete"]=="Labour"
LAB2019 = pan_dataset_votes["2019GEVote_complete"]=="Labour"


In [501]:
(LAB2015&LD05_10).sum()/(LAB2015&LD05_10_tot).sum()

0.11851465117796776

In [503]:
(LAB2015&LD05_10).sum()/(LAB2015&LD05_10_tot).sum()

0.35422623813336385

In [505]:
(LAB2017&LD05_10).sum()/(LAB2017&LD05_10_tot).sum()

0.31912166304408407

In [506]:
(LAB2019&LD05_10).sum()/(LAB2019&LD05_10_tot).sum()

0.3342492322611928